In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
columns = ['Event', 'Promotion', 'Date', 'Venue', 'City', 'Attendance', 'Gate', 'Buyrate']
wiki_df = pd.read_csv("data/ufc_data.csv", names=columns)
wiki_df.head()

,Event,Promotion,Date,Venue,City,Attendance,Gate,Buyrate
0,UFC 1: The Beginning,Ultimate Fighting Championship,"November 12, 1993",McNichols Sports Arena,"Denver, Colorado","7,800[1]",NaN,"86,000[2]"
1,UFC 2: No Way Out,Ultimate Fighting Championship,"March 11, 1994",Mammoth Gardens,"Denver, Colorado","2,000",NaN,NaN
2,UFC 3: The American Dream,Ultimate Fighting Championship,"September 9, 1994[1]",Grady Cole Center,"Charlotte, North Carolina","3,000",NaN,"90,000"
3,UFC 4: Revenge of the Warriors,Ultimate Fighting Championship,"December 16, 1994",Expo Square Pavilion,"Tulsa, Oklahoma","5,857",NaN,"120,000"
4,UFC 5: Gracie vs. Shamrock 2,Ultimate Fighting Championship,"April 7, 1995",Independence Arena,"Charlotte, North Carolina","6,000",NaN,"260,000[1]"


In [3]:
wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 8 columns):
Event         441 non-null object
Promotion     441 non-null object
Date          441 non-null object
Venue         441 non-null object
City          441 non-null object
Attendance    392 non-null object
Gate          307 non-null object
Buyrate       178 non-null object
dtypes: object(8)
memory usage: 27.6+ KB


## Clean Date

In [4]:
wiki_df['Date'].unique()

array(['November 12, 1993', 'March 11, 1994', 'September 9, 1994[1]',
       'December 16, 1994', 'April 7, 1995', 'July 14, 1995',
       'September 8, 1995', 'December 16, 1995', 'February 16, 1996',
       'May 17, 1996', 'July 12, 1996', 'September 20, 1996',
       'December 7, 1996', 'February 7, 1997', 'May 30, 1997',
       'July 27, 1997', 'October 17, 1997', 'December 21, 1997',
       'March 13, 1998', 'May 15, 1998', 'October 16, 1998',
       'January 8, 1999', 'March 5, 1999', 'May 7, 1999', 'July 16, 1999',
       'September 24, 1999', 'November 19, 1999', 'March 10, 2000',
       'April 14, 2000', 'June 9, 2000', 'September 22, 2000',
       'November 17, 2000', 'December 16, 2000', 'February 23, 2001',
       'May 4, 2001', 'June 29, 2001', 'September 28, 2001',
       'November 2, 2001', 'January 11, 2002', 'March 22, 2002',
       'May 10, 2002', 'June 22, 2002', 'July 13, 2002',
       'September 27, 2002', 'November 22, 2002', 'February 28, 2003',
       'April 25,

In [5]:
import re

date_p1 = re.compile("([A-Z][a-z]*) ([0-9]+), ([0-9]+)")
date_p2 = re.compile("([0-9]+) ([A-Z][a-z]*) ([0-9]+)")

In [6]:
from datetime import datetime

In [7]:
def clean_date(date):

    date = date.replace("\xa0", ' ')
    
    if date_p1.match(date):
        chunk = date_p1.findall(date)[0]
        clean_string = "{} {} {}".format(chunk[0], chunk[1], chunk[2])    
    elif date_p2.match(date):
        chunk = date_p2.findall(date)[0]
        clean_string = "{} {} {}".format(chunk[1], chunk[0], chunk[2])
    elif date == 'Cancelled':
        return date
        
    dt_object = datetime.strptime(clean_string, "%B %d %Y").date()
    return dt_object
    

In [8]:
wiki_df['Date'] = wiki_df['Date'].apply(clean_date)

In [9]:
wiki_df = wiki_df[wiki_df['Date'] != 'Cancelled']

## Clean Buyrate

In [10]:
wiki_df['Buyrate'].unique()

array(['86,000[2]', nan, '90,000', '120,000', '260,000[1]', '240,000[1]',
       '190,000[1]', '251,000', '300,000[1]', '141,000', '96,000',
       '92,000', '122,000', '75,000', '65,000', '35,000', '55,000[1]',
       '50,000', '45,000', '100,000', '60,000', '49,000', '94,000',
       '40,000', '80,000[1]', '105,000[1]', '110,000[1]', '105,000',
       '280,000[1]', '150,000', '125,000', '200,000', '400,000[2]',
       '300,000[2]', '425,000[2]', '620,000[1]', '775,000 [1]',
       '500,000[2]', '500,000[3]', '1,050,000[2]', '350,000[3]',
       '534,000[2]', '400,000[citation needed]', '675,000[2]',
       '200,000[2]', '425,000', '520,000[2]', '475,000[1]', '325,000[1]',
       '400,000', '650,000[1]', '225,000', '600,000', '325,000',
       '530,000', '475,000', '215,000[2]', '540,000', '625,000[1]',
       '480,000 [1]', '1,010,000[3]', '1,000,000 [1]', '350,000 [1]',
       '920,000[2]', '350,000', '635,000[2]', '360,000[2]',
       '1,600,000[2]', '850,000[2]', '435,000[2]', '37

In [11]:
buy_p1 = re.compile("([0-9]+,[0-9]+,*[ ]*[0-9]*)")

In [12]:
def clean_buyrate(buyrate):
    try:
        if buy_p1.match(buyrate):
            chunk = buy_p1.findall(buyrate)[0]
            clean_string = "{}".format(chunk).replace(',','')
            return int(clean_string)
        else:
            print(buyrate)
    except TypeError:
        pass # so we don't print out NaNs

In [13]:
wiki_df['Buyrate'] = wiki_df['Buyrate'].apply(clean_buyrate)

$2,028,000[1]


$2,028,000[1] converted to NaN in Buyrate_2

## Clean Gate

In [14]:
wiki_df['Gate'].unique()

array([nan, '$816,660', '$502,550', '$898,850[1]', '$1,540,000',
       '$645,140', '$127,985', '$1,377,000[1]', '$1,444,020[1]',
       '$901,655[1]', '$1,293,035[1]', '$1,493,000', '$2,575,450',
       '$1,100,000', '$2,336,000', '$1,986,000', '$144,600[1]',
       '$3,382,400', '$1,758,450', '$199,150[1]', '$2,191,450',
       '$2,900,090', '$134,368[2]', '$3,350,775.15', '$3,040,880[1]',
       '$1,582,370[1]', '$1,790,490[1]', '$2,138,020[2]', '$5,397,300[1]',
       '$2,767,130[2]', '$3,014,000[1]', '$2,817,200[2]',
       '$2,628,472 (£1.3 million GBP)[2]', '$4,304,000[1]',
       '£700,000 (approx. $1.2 million)[1]', '$526,000[12]',
       '$1,551,920[1]', '$3,307,000[1]', '$2.7 million[1]', '$270,000[1]',
       '$1,985,000', '$2,540,000', '$2,100,000', '$4,994,000',
       '$1,250,000', '$2,437,890', '$2,200,000[1]', '$753,429[2]',
       '$5,100,000', '$3,732,000[1]', '2,920,000', '$495,350',
       '$3,350,730', '$2,252,000', '$2,600,000', '$700,000', '$2,850,000',
       '

In [15]:
gate_p1 = re.compile("\$([0-9]+,[0-9]*,*[0-9]*)")
gate_p2 = re.compile("\$([0-9]\.[0-9]+)")

In [16]:
example = wiki_df['Gate'][50]
example

'$1,377,000[1]'

In [17]:
def clean_gate(gate):
    try:
        if gate_p1.match(gate):
            chunk = gate_p1.findall(gate)[0]
            clean_string = "{}".format(chunk).replace(',','')
            return int(clean_string)
        elif gate_p2.match(gate):
            chunk = gate_p2.findall(gate)
            gate_num = float(chunk) * 1000000
            print(gate_num)
            return int(gate_num)
        else:
            print(gate)
    except TypeError:
        pass # so we don't print out NaNs

In [18]:
wiki_df['Gate'] = wiki_df['Gate'].apply(clean_gate)

£700,000 (approx. $1.2 million)[1]
2,920,000
C$995,624[1]
2,200,000 AUD[1]
$ 2,385,230[1]


## Clean Attendance

In [19]:
wiki_df['Attendance'].unique()

array(['7,800[1]', '2,000', '3,000', '5,857', '6,000', '2,700', '9,000',
       '2,800', '13,000', '10,000', '4,300', '4,500', '3,100', '5,100',
       '5,000', nan, '4,600', '1,414[1]', '12,500', '9,500', '9,600',
       '10,000[1]', '7,200', '3,700', '3,800', '7,800', '13,265',
       '13,401', '6,700', '9,800', '10,400', '9,200', '10,700[1]',
       '11,437[1]', '10,000 (Paid: 6,528)[1]', '12,100 (paid 8,597)[1]',
       '11,072 (Paid: 9,268)[1]', '14,562 (Paid: 12,643)', '12,000',
       '13,520 (Paid: 11,634)[1]', '8,000', '12,000 (Paid: 9,995)[1]',
       '1,008', '11,000 (Paid: 10,301)[1]', '9,569 (8,183 paid)[1]',
       '843', '13,814 (13,060 paid)[1]', '14,802 (10,347 paid)', '954[1]',
       '11,167 (9,999 paid)', '9,859 (8,954 paid)[1]',
       '12,604 (9,343 paid)[1]', '3,510', '10,173 (8,913 paid)[1]',
       '14,666 (12,362 paid)[1]', '13,761 (12,191 paid)[1]',
       '10,227 (8,700 paid)[1]', '19,049 (17,358 paid)', '1,734',
       '15,269 (12,516 paid)[1]', '15,114 (12

In [20]:
attend_p1 = re.compile("([0-9]+,*[0-9]*)")

In [21]:
example = wiki_df['Attendance'][80]
example

'14,666 (12,362 paid)[1]'

In [22]:
attend_p1.match(example)

<_sre.SRE_Match object; span=(0, 6), match='14,666'>

In [23]:
def clean_attendance(attend):
    if '♠' in str(attend):
        attend = attend.split('♠')[1]
    try:
        if attend_p1.match(attend):
            chunk = attend_p1.findall(attend)[0]
            clean_string = chunk.replace(',','')
            return int(clean_string)
        else:
            print(attend)
    except TypeError:
        pass # so we don't print out NaNs

In [24]:
wiki_df['Attendance'] = wiki_df['Attendance'].apply(clean_attendance)

In [25]:
wiki_df = wiki_df.set_index(pd.DatetimeIndex(wiki_df['Date']))
wiki_df.drop('Date', axis=1, inplace=True)

In [26]:
wiki_df

,Event,Promotion,Venue,City,Attendance,Gate,Buyrate
Date,,,,,,,
1993-11-12,UFC 1: The Beginning,Ultimate Fighting Championship,McNichols Sports Arena,"Denver, Colorado",7800.0,NaN,86000.0
1994-03-11,UFC 2: No Way Out,Ultimate Fighting Championship,Mammoth Gardens,"Denver, Colorado",2000.0,NaN,NaN
1994-09-09,UFC 3: The American Dream,Ultimate Fighting Championship,Grady Cole Center,"Charlotte, North Carolina",3000.0,NaN,90000.0
1994-12-16,UFC 4: Revenge of the Warriors,Ultimate Fighting Championship,Expo Square Pavilion,"Tulsa, Oklahoma",5857.0,NaN,120000.0
1995-04-07,UFC 5: Gracie vs. Shamrock 2,Ultimate Fighting Championship,Independence Arena,"Charlotte, North Carolina",6000.0,NaN,260000.0
1995-07-14,UFC 6: Shamrock vs. Severn,Ultimate Fighting Championship,Casper Events Center,"Casper, Wyoming",2700.0,NaN,240000.0
1995-09-08,UFC 7,Ultimate Fighting Championship,Memorial Auditorium,"Buffalo, New York",9000.0,NaN,190000.0
1995-12-16,Ultimate Ultimate 1995,Ultimate Fighting Championship,Mammoth Gardens,"Denver, Colorado",2800.0,NaN,251000.0
1996-02-16,UFC 8: Shamrock vs. Leopoldo,Ultimate Fighting Championship,Ruben Rodriguez Coliseum,"Bayamón, Puerto Rico",13000.0,NaN,300000.0


# Fill missing Buyrates with Tapology dataset

In [27]:
tap_df = pd.read_csv("data/tapology_data.csv")
tap_df.head(10)

,Event,Title,Date,Buyrate
0,UFC 202,Diaz vs. McGregor 2,2016.08.20,"\n1,600,000\n"
1,UFC 196,McGregor vs. Diaz,2016.03.05,"\n1,317,000\n"
2,UFC 205,Alvarez vs. McGregor,2016.11.12,"\n1,300,000\n"
3,UFC 100,NaN,2009.07.11,"\n1,300,000\n"
4,UFC 194,Aldo vs. McGregor,2015.12.12,"\n1,200,000\n"
5,UFC 116,Lesnar vs Carwin,2010.07.03,"\n1,160,000\n"
6,UFC 193,Rousey vs. Holm,2015.11.14,"\n1,100,000\n"
7,UFC 207,Nunes vs. Rousey,2016.12.30,"\n1,100,000\n"
8,UFC 92,The Ultimate 2008,2008.12.27,"\n1,050,000\n"
9,UFC 114,Jackson vs Evans,2010.05.29,"\n1,050,000\n"


In [28]:
def clean_tap_buyrate(buyrate):
    return int(buyrate.replace(',', '').replace('\n', ''))

In [29]:
tap_df['Buyrate'] = tap_df['Buyrate'].apply(clean_tap_buyrate)

In [30]:
def clean_tap_date(date):
    return datetime.strptime(date, "%Y.%m.%d").date()

In [31]:
tap_df['Date'] = tap_df['Date'].apply(clean_tap_date)

In [32]:
tap_df = tap_df.set_index(pd.DatetimeIndex(tap_df['Date']))

In [33]:
tap_df.head(10)

,Event,Title,Date,Buyrate
Date,,,,
2016-08-20,UFC 202,Diaz vs. McGregor 2,2016-08-20,1600000
2016-03-05,UFC 196,McGregor vs. Diaz,2016-03-05,1317000
2016-11-12,UFC 205,Alvarez vs. McGregor,2016-11-12,1300000
2009-07-11,UFC 100,NaN,2009-07-11,1300000
2015-12-12,UFC 194,Aldo vs. McGregor,2015-12-12,1200000
2010-07-03,UFC 116,Lesnar vs Carwin,2010-07-03,1160000
2015-11-14,UFC 193,Rousey vs. Holm,2015-11-14,1100000
2016-12-30,UFC 207,Nunes vs. Rousey,2016-12-30,1100000
2008-12-27,UFC 92,The Ultimate 2008,2008-12-27,1050000


In [34]:
wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 438 entries, 1993-11-12 to 2018-12-29
Data columns (total 7 columns):
Event         438 non-null object
Promotion     438 non-null object
Venue         438 non-null object
City          438 non-null object
Attendance    392 non-null float64
Gate          280 non-null float64
Buyrate       177 non-null float64
dtypes: float64(3), object(4)
memory usage: 27.4+ KB


In [35]:
wiki_df['Buyrate'] = wiki_df['Buyrate'].fillna(tap_df['Buyrate'])

In [36]:
wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 438 entries, 1993-11-12 to 2018-12-29
Data columns (total 7 columns):
Event         438 non-null object
Promotion     438 non-null object
Venue         438 non-null object
City          438 non-null object
Attendance    392 non-null float64
Gate          280 non-null float64
Buyrate       190 non-null float64
dtypes: float64(3), object(4)
memory usage: 27.4+ KB


In [37]:
wiki_df

,Event,Promotion,Venue,City,Attendance,Gate,Buyrate
Date,,,,,,,
1993-11-12,UFC 1: The Beginning,Ultimate Fighting Championship,McNichols Sports Arena,"Denver, Colorado",7800.0,NaN,86000.0
1994-03-11,UFC 2: No Way Out,Ultimate Fighting Championship,Mammoth Gardens,"Denver, Colorado",2000.0,NaN,300000.0
1994-09-09,UFC 3: The American Dream,Ultimate Fighting Championship,Grady Cole Center,"Charlotte, North Carolina",3000.0,NaN,90000.0
1994-12-16,UFC 4: Revenge of the Warriors,Ultimate Fighting Championship,Expo Square Pavilion,"Tulsa, Oklahoma",5857.0,NaN,120000.0
1995-04-07,UFC 5: Gracie vs. Shamrock 2,Ultimate Fighting Championship,Independence Arena,"Charlotte, North Carolina",6000.0,NaN,260000.0
1995-07-14,UFC 6: Shamrock vs. Severn,Ultimate Fighting Championship,Casper Events Center,"Casper, Wyoming",2700.0,NaN,240000.0
1995-09-08,UFC 7,Ultimate Fighting Championship,Memorial Auditorium,"Buffalo, New York",9000.0,NaN,190000.0
1995-12-16,Ultimate Ultimate 1995,Ultimate Fighting Championship,Mammoth Gardens,"Denver, Colorado",2800.0,NaN,251000.0
1996-02-16,UFC 8: Shamrock vs. Leopoldo,Ultimate Fighting Championship,Ruben Rodriguez Coliseum,"Bayamón, Puerto Rico",13000.0,NaN,300000.0


In [38]:
wiki_df.to_csv("ufc_clean.csv")